# Keeping it Clean: Building a Spam Filter with Naïve Bayes

## Table of Contents

* [Introduction](#intorduction)
* [Goal](#goal)
* [Summary](#summary)
* [The Data](#the_data)
* [Training and Test Set](#training_and_test_set)
* [Data Cleaning](#data_cleaning)
  * [Letter Case and Punctuation](#letter_case_and_punctuation)
  * [Creating the Vocabulary](#creating_the_vocabulary)
  * [The Final Training Set](#final_training_set)
* [Training the Model](#training_the_model)
  * [Calculating Constants First](#calculating_constants_first)
  * [Calculating Parameters](#calculating_parameters)
* [Classifying A New Message](#classifying_a_new_message)
* [Measuring the Spam Filter's Accuracy](#measuring_the_spam_filters_accuracy)
* [Conclusion](#conclusion)

## Introduction  <a name="introduction"></a>

The Naïve Bayes algorithm is a simple technique for constructing classifiers.
In this guided project, we're going to study the practical side of the algorithm by building a spam filter for SMS messages.
Naïve Bayes is especially effective for text classification tasks like spam filtering because it handles high-dimensional feature spaces well and performs surprisingly well even with its 'naïve' assumption of feature independence. The algorithm is also computationally efficient and requires relatively little training data to achieve good results, making it ideal for processing short text messages. Additionally, its probabilistic nature allows us to easily interpret and understand why certain messages are classified as spam or legitimate.

## Goal  <a name="goal"></a>

We aim to write a program that will classify new SMS messages as spam or ham with an accuracy greater than 80%.
We'll calculate this accuracy by comparing our model's predictions against known classifications, determining the percentage of messages where our prediction matches the original label.


## Summary  <a name="summary"></a>

We managed to create a spam filter with an accuracy of 98.7%. 

## The Data <a name="the_data"></a>

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). You can also download the dataset directly from [this link](https://dq-content.s3.amazonaws.com/433/SMSSpamCollection). The data collection process is described in more details on [this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition), where you can also find some of the authors' papers.

We'll start by reading the dataset and printing some initial data about it: 

In [5]:
import pandas as pd
import re
from numpy import prod

collection = pd.read_csv('SMSSpamCollection', sep='\t', header=None, names=['Label', 'SMS'])

print("The number of rows and columns in the dataset:")
print(collection.shape)

collection.head()

The number of rows and columns in the dataset:
(5572, 2)


,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
collection['Label'].value_counts()

Label
ham     4825
spam     747
Name: count, dtype: int64

In [7]:
collection['Label'].value_counts(normalize=True)

Label
ham     0.865937
spam    0.134063
Name: proportion, dtype: float64

This is a very simple dataset. It's composed of only two columns - the contents of the message (`SMS`) and a classification (`Label`. `spam` for spam messages, `ham` for legitimate ones). About 87% of the messages are legitimate, and the remaining 13% is spam.

## Training and Test Set <a name="training_and_test_set"></a>

Before building and training our spam filter, we should split our dataset into a training set and a test set. The **training set** is used to "train" the computer how to classify messages, and the **test set** is used to test how good the spam filter is with classifying new messages.

We're going to keep 80% of our dataset for training, and 20% for testing (we want to train the algorithm on as much data as possible, but we also want to have enough test data). The dataset has 5,572 messages, which means that:

* The training set will have 4,458 messages (about 80% of the dataset).
* The test set will have 1,114 messages (about 20% of the dataset).

Since all messages in the dataset are already classified, the way to test our algorithm is by running it on the test set and comparing its classifications against the actual classification.
As mentioned above, our goal is to create a classifier with an accuracy greater than 80%, so when the algorithm correctly specifies over 80% of the test set we'll consider the goal reached.

To create both subsets we'll start by randomizing the entire dataset to ensure that spam and ham messages are spread properly throughout.

In [8]:
randomized_collection = collection.sample(frac=1, random_state=1)

Now we can safely split the dataset into a training and a test set:

In [9]:
training_set = randomized_collection.iloc[:4458].copy().reset_index(drop=True)
test_set = randomized_collection.iloc[4458:].copy().reset_index(drop=True)

Division of ham and spam messages in the Training set:

In [10]:
training_set['Label'].value_counts(normalize=True)

Label
ham     0.86541
spam    0.13459
Name: proportion, dtype: float64

Division of ham and spam messages in the Test set:

In [11]:
test_set['Label'].value_counts(normalize=True)

Label
ham     0.868043
spam    0.131957
Name: proportion, dtype: float64

Good, both sets have a similar ham/spam percentage. The difference between them is negligible.

## Data Cleaning <a name="data_cleaning"></a>

### Letter Case and Punctuation <a name="letter_case_and_punctuation"></a>

Before we can use the training we'll need to perform a bit of data cleaning to bring the data in a format that will allow us to easily extract all the information we need.
To make calculations easiter we want to add columns to the data set (one per word in the vocabulary) with counters how many times each word appeared in each message.

Let's begin the data cleaning process by removing the punctiation and bringing all the words to lower case:

In [12]:
training_set['clean_SMS'] = training_set['SMS'].str.replace('\W', ' ').str.lower()

In [13]:
training_set['clean_SMS']

0                            yep, by the pretty sculpture
1           yes, princess. are you going to make me moan?
2                              welp apparently he retired
3                                                 havent.
4       i forgot 2 ask ü all smth.. there's a card on ...
                              ...                        
4453    sorry, i'll call later in meeting any thing re...
4454    babe! i fucking love you too !! you know? fuck...
4455    u've been selected to stay in 1 of 250 top bri...
4456    hello my boytoy ... geeee i miss you already a...
4457                             wherre's my boytoy ? :-(
Name: clean_SMS, Length: 4458, dtype: object

### Creating the Vocabulary <a name="creating_the_vocabulary"></a>

To create a column for each word in our vocabulary, we'll need to create a vocabulary. We'll do that by reading each SMS message and adding each word to a `vocabulary` set.

In [14]:
# split each label into words, then iterate over each list of words
# and add all the words in that list to the vocabulary
training_set['clean_SMS'] = training_set['clean_SMS'].str.split()
vocabulary = set([word for words in training_set['clean_SMS'] for word in words])

In [15]:
len(vocabulary)

11860

There are 7,783 unique words in our vocabulary.

### The Final Training Set <a name="final_training_set"></a>

Now it's time to create the words columns (one column for every word in our dictionary). We'll do that by first creating a word count dictionary, then building a dataframe using that dictionary. 

In [16]:
# Create an dictionary of lists containing zeros. Each list will match a single word in our vocabulary
# and its length will be the size of our training set
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

# Now we populate the lists
for index, sms in enumerate(training_set['clean_SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1

In [17]:
# Create a dataframe from the dictionary
word_counts = pd.DataFrame(word_counts_per_sms)

In [18]:
# Concatenate with the training set, to get the Label, SMS and clean_SMS columns
word_counts = pd.concat([training_set, word_counts], axis=1)

In [19]:
word_counts.head()

,Label,SMS,clean_SMS,much!!i,lane,me....,said,unique&i,shoppin,include,...,sweetie,next.,tears,college?,2yr,5years,will!,hurried,vote.,:-)only
0,ham,"Yep, by the pretty sculpture","[yep,, by, the, pretty, sculpture]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"Yes, princess. Are you going to make me moan?","[yes,, princess., are, you, going, to, make, m...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,Welp apparently he retired,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,Havent.,[havent.],0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,I forgot 2 ask ü all smth.. There's a card on ...,"[i, forgot, 2, ask, ü, all, smth.., there's, a...",0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Training the Model <a name="training_the_model"></a>

Now that we're done with data cleaning and have a training set to work with, we can begin creating the spam filter.

### Calculating Constants First <a name="calculating_constants_first"></a>

the Naïve Bayes algorithm will need to know the probability values of the two equations below to be able to classify new messages:

$$P \left(Spam \middle| \ w_1, w_2, ..., w_n\right) \propto P(Spam)\cdot\prod_{i=1}^{n} P\left( w_i \middle| \, Spam \right)$$

$$P \left(Ham \middle| \ w_1, w_2, ..., w_n\right) \propto P(Ham)\cdot\prod_{i=1}^{n} P\left( w_i \middle| \, Ham \right)$$

Also, to calculate P(w<sub>i</sub>|Spam) and P(w<sub>i</sub>|Ham) inside the formulas above, we need to use these equations:

$$\dfrac{N_{w_i|Spam}+\alpha}{N_{Spam}+\alpha \cdot N_{Vocabulary}
}$$


$$\dfrac{N_{w_i|Ham}+\alpha}{N_{Ham}+\alpha \cdot N_{Vocabulary}
}$$

Let's start by calculating P(Spam), P(Ham), N<sub>Spam</sub>, N<sub>Ham</sub> and N<sub>Vocabulary</sub> from the training set:

In [20]:
# P(Spam) and P(Ham)
p_spam = word_counts['Label'].value_counts(normalize=True)['spam']
p_ham = 1 - p_spam

# N(Spam): The number of words in all spam messages
n_spam = word_counts[word_counts['Label'] == 'spam']['clean_SMS'].apply(len).sum()

# N(Ham): The number of words in all ham messages
n_ham = word_counts[word_counts['Label'] == 'ham']['clean_SMS'].apply(len).sum()

# N(Vocabulary): The size of our vocabulary
n_vocab = len(vocabulary)

# Set Laplace smoothing alpha as 1
alpha = 1

print("P(Spam): {:.3}".format(p_spam))
print("P(Ham): {:.3}".format(p_ham))
print("N(Spam): {}".format(n_spam))
print("N(Ham): {}".format(n_ham))
print("N(Vocabulary): {}".format(n_vocab))

P(Spam): 0.135
P(Ham): 0.865
N(Spam): 14257
N(Ham): 55376
N(Vocabulary): 11860


### Calculating Parameters <a name="calculating_parameters"></a>

To same time during the classification process we'll calculate the probability values for each word in our vocabulary ahead of time. That way we can use the calculated value instead of recalculating for each new message.

In [21]:
# Isolate the spam and ham messages
spam_word_count = word_counts[word_counts['Label'] == 'spam'].copy()
ham_word_count = word_counts[word_counts['Label'] == 'ham'].copy()

In [22]:
# Populate two dictionaries - one for spam words and the other for ham words
spam_den = n_spam + alpha * n_vocab
ham_den = n_ham + alpha * n_vocab

spam_params = {word:(spam_word_count[word].sum() + alpha) / spam_den for word in vocabulary}
ham_params = {word:(ham_word_count[word].sum() + alpha) / ham_den for word in vocabulary}

## Classifying A New Message <a name="classifying_a_new_message"></a>

Now that we've calculated all the constants and parameters we need, we can start creating the spam filter. The spam filter can be understood as a function that:

* Takes in as input a new message (w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>)
* Calculates P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>)
* Compares the values of P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) and P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), and:
* If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) > P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as ham.
* If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) < P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the message is classified as spam.
* If P(Ham|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>) = P(Spam|w<sub>1</sub>, w<sub>2</sub>, ..., w<sub>n</sub>), then the algorithm may request human help.

In [23]:
def classify(message:str, print_output = True) -> str:
    """
    Classify a message as spam or ham using Naive Bayes classification.
    
    Processes the input message by removing punctuation, converting it to lowercase,
    and tokenizing it into words. Then applies Naive Bayes classification using
    pre-calculated word probabilities to determine if the message is spam or ham.
    
    Args:
       message (str): The text message to classify
       print_output (bool, optional): Whether to print probability details. Defaults to True
       
    Returns:
       str: Classification result - either 'spam', 'ham', or 'undecided'
       
    Examples:
       >>> classify("Win a free iPhone!")
       Classification: spam
       'spam'
       
       >>> classify("Meeting at 3pm tomorrow", print_output=False)
       'ham'
    
    Notes:
       - Uses global variables:
           * p_spam: Prior probability of spam
           * p_ham: Prior probability of ham
           * spam_params: Dictionary of word probabilities in spam messages
           * ham_params: Dictionary of word probabilities in ham messages
       - Text preprocessing steps:
           1. Removes all non-word characters
           2. Converts to lowercase
           3. Splits into individual words
       - Uses product of word probabilities following Naive Bayes assumption
       - Returns 'undecided' if spam and ham probabilities are equal
       
    Dependencies:
       - re: For regular expression text preprocessing
       - numpy.prod: For calculating product of probabilities
       
    See Also:
       numpy.prod: Used to multiply probabilities
    """

    message = (re.sub('\W', ' ', message) # Remove punctuations
              .lower() # Move to lowercase
              .split() # separate into words
              )

    # Calculate the odds of the message being spam or ham
    p_spam_given_message = p_spam * prod([spam_params[word]
                                          for word in message
                                          if word in spam_params])
    
    p_ham_given_message = p_ham * prod([ham_params[word]
                                        for word in message
                                        if word in ham_params])

    
    if p_ham_given_message > p_spam_given_message:
        result = 'ham'
    elif p_ham_given_message < p_spam_given_message:
        result = 'spam'
    else:
        result = 'undecided'
    
    if print_output:
        print('P(Spam|message):', p_spam_given_message)
        print('P(Ham|message):', p_ham_given_message)

        print('Classification: {}'.format(result))
    
    return result

Let's test our code on two messages. An obvious spam and an obvious ham:

In [24]:
classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 1.168002363207846e-26
P(Ham|message): 6.088544142463393e-28
Classification: spam


'spam'

In [25]:
classify("Sounds good, Tom, then see u there")

P(Spam|message): 2.2342992839679443e-26
P(Ham|message): 8.376346103813855e-22
Classification: ham


'ham'

Okay, looks like it's working fine. Now it's time to measure the filter's accuracy.

## Measuring the Spam Filter's Accuracy <a name="measuring_the_spam_filters_accuracy"></a>

In [26]:
# Run the filter on the test set
test_set['predicted'] = test_set['SMS'].apply(classify, print_output=False)

In [27]:
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now all that's lef tis to compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages:

In [28]:
test_set['correct'] = test_set['Label'] == test_set['predicted']

# Accuracy is measured by dividing the number of correct predictions by the total number of messages
accuracy = test_set['correct'].sum() / test_set.shape[0]

What's our accuracy?

In [29]:
print(f"{accuracy*100:.03}%")

97.8%


Amazing. Our goal was to create a filter with above 80% success rate, and our filter has a success rate of 97.8%. Meaning only 13 messages out of 1,000 will be misclassified.

But let's look at some more metrics to better assess our model.

In [54]:
precision = test_set[(test_set['predicted'] == 'spam') & (test_set['Label'] == 'spam')]['predicted'].count() / test_set[test_set['predicted'] == 'spam']['predicted'].count()
recall = test_set[(test_set['predicted'] == 'spam') & (test_set['Label'] == 'spam')]['predicted'].count() / test_set[test_set['Label'] == 'spam']['predicted'].count()
f1_score = 2 * (precision * recall) / (precision + recall)

print(f"Precision = {precision*100:.03}%")
print(f"Recall = {recall*100:.03}%")
print(f"F1 Score = {f1_score*100:0.3}%")

Precision = 94.3%
Recall = 89.8%
F1 Score = 92.0%


In [55]:
def confusion_matrix(df: pd.DataFrame, col1: str, col2: str) -> pd.DataFrame:
    """
    Given a dataframe with at least
    two categorical columns, create a 
    confusion matrix of the count of the columns
    cross-counts
    
    Example:
    
    confusion_matrix(test_df, 'predicted', 'actual')
    """
    return (
            df
            .groupby([col1, col2])
            .size()
            .unstack(fill_value=0)
            )

Finally, the model's confusion matrix:

In [56]:
cm = confusion_matrix(test_set, 'predicted', 'Label')
cm

Label,ham,spam
predicted,,
ham,958,15
spam,8,132
undecided,1,0


The results show strong overall performance but with some interesting nuances:

1. Precision of 94.3% indicates that when our model predicts spam, it's right about 94 times out of 100. The relatively high precision is crucial for a spam filter, as false positives (marking legitimate emails as spam) can be particularly problematic for users.

2. Recall of 89.8% shows we're catching about 90% of all actual spam messages. Looking at the confusion matrix, we missed 15 spam messages (labelled them as ham) out of the 147 total spam messages.

3.  The confusion matrix reveals:
    * Very few false positives: only 8 ham messages were incorrectly labelled as spam
    * A slightly higher number of false negatives: 15 spam messages got through
    * One message remained "undecided," which is better than making a wrong classification
    * The model handled ham messages particularly well, correctly identifying 958 out of 967 (99.1%)

Key takeaway: The model is performing well but appears to be slightly conservative in its spam classification, preferring to let a spam message through rather than risk blocking legitimate messages. This is generally a good approach for a spam filter, as users typically prefer to receive occasional spam rather than miss important legitimate messages.

## Conclusion <a name="conclusion"></a>

We used The Naïve Bayes algorithm to build a spam filter, achieving strong overall performance with 94.3% precision and 89.8% recall.
The confusion matrix reveals that our model is notably conservative in its classifications, preferring to let a spam message through rather than risk blocking legitimate messages. 
While this conservative approach aligns well with user preferences — most people would rather receive occasional spam than miss important legitimate messages — there's still room for improvement.

Several promising avenues could enhance our system further. We could explore more sophisticated feature extraction methods, such as incorporating word embeddings or n-grams to capture contextual relationships between words. This might help catch the small amount of spam messages that slipped through while maintaining our strong precision.
We might also experiment with different algorithms like Support Vector Machines or modern deep learning approaches to compare their performance against our Naive Bayes implementation.

The dataset itself presents opportunities for improvement. Expanding it with more recent spam examples would help the model stay current with evolving spam tactics. Additionally, collecting messages in multiple languages or from different communication platforms would make the classifier more robust and widely applicable.

Finally, we could enhance the preprocessing stage by implementing more nuanced text cleaning methods, such as lemmatization instead of basic stemming, or by preserving certain structural features of messages that might be indicative of spam. While our current model performs admirably, with an F1 score of 92%, these refinements could lead to an even more reliable and versatile spam detection system.